In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline
import tensorflow_hub as thub
import tensorflow.keras.backend as K

In [2]:
!pip install adversarial-robustness-toolbox
import art

     |████████████████████████████████| 1.1 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 37.3 MB/s 
     |████████████████████████████████| 26.3 MB 92 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [3]:
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod
from art.attacks.extraction import CopycatCNN, FunctionallyEquivalentExtraction, KnockoffNets

In [4]:
def sample_by_class(data, labels, num_samples=100):
    sample_data = []
    sample_labels = []
    unq_labels = list(range(labels.shape[1]))
    for label in unq_labels:
        idx = labels[:,label]==1
        sample_set = data[idx][0:num_samples].copy()
        label_set = labels[idx][0:num_samples].copy()
        sample_data.append(sample_set)
        sample_labels.append(label_set)
    
    sample_data = np.concatenate(sample_data)
    sample_labels = np.concatenate(sample_labels)
    print(sample_data.shape, sample_labels.shape)
    return sample_data, sample_labels

def subset_data(data, labels, fraction=5):
    data_size = data.shape[0]
    out_size = int(data_size*fraction/100)
    idx = np.random.choice(data_size, out_size, replace=False)
    out_data = data[idx].copy()
    out_labels = labels[idx].copy()
    
    print(out_data.shape, out_labels.shape)
    return out_data, out_labels

In [5]:
from tensorflow import keras
(x_train,y_train),(x_test,y_test) = keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 5s 0us/step


In [6]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
testd = x_train[0:1000].copy()
testl = y_train[0:1000].copy()

In [8]:
def build_ganeval_model():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(tf.keras.layers.Activation('softmax'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay= 1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [9]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
K.clear_session()
model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
ganeval_module = thub.Module(model_url)

In [10]:
ge_cifar_clf = tf.keras.Sequential()
ge_cifar_clf.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
# gan_eval_model.add(gan_eval_layer)
ge_cifar_clf.add(thub.KerasLayer(ganeval_module))
ge_cifar_clf.add(tf.keras.layers.Activation('softmax'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
ge_cifar_clf.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                       loss=tf.keras.losses.CategoricalCrossentropy(),
                       metrics=['accuracy'])
ge_cifar_clf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 10)                7796426   
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [12]:
ge_cifar_clf = build_ganeval_model()
ge_cifar_clf.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 10)                7796426   
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [13]:
ge_cifar_clf.evaluate(testd, testl)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[166.622517578125, 0.11]

In [14]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.4)

In [15]:
ge_cifar_clf.evaluate(testd, testl)

[166.622517578125, 0.11]

In [16]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.4)

In [17]:
x_test_adv = attack_fgsm.generate(testd.astype(np.float32).copy())

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [18]:
ge_cifar_clf.evaluate(x_test_adv, testl)

[166.08971728515624, 0.11]

In [19]:
def build_substitute_model(enable_logits=True):
    model = tf.keras.Sequential( )
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32,32,3)))
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    if enable_logits:
        model.add(Activation("linear"))
    else:
        model.add(Activation("softmax"))
        
    model.compile(loss=CategoricalCrossentropy(from_logits=enable_logits),
                  optimizer='adam',
                  metrics=['accuracy'] )
    
    return model

In [20]:
from tensorflow.keras.layers import Conv2D
from keras.layers import Activation, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout,Flatten
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
sub_model = build_substitute_model()
sub_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_2 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                      

In [33]:
def extraction_attack(ART_attack, x_train, y_train, fractions, epoch=30, verbose=0):
    
    data_size = x_train.shape[0]
    epochs = epoch
    
    ### Train and Test sample data from each class to get final results
    print("Extract 100 Training samples from each class:")
    x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
    print("Extract 100 Test samples from each class:")
    x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)
    
    ge_cifar_clf = build_ganeval_model()
    cloud_art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
    
    loss1, acc1 = cloud_art_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
    loss2, acc2 = cloud_art_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
    
    clf_results = []
    
    for each_frac in fractions:
        max_queries = int(data_size*each_frac/100)
        print("Attacking the victim with %d percent of training data: %d queries..."%(each_frac, max_queries))
#         partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=each_frac)
        
        sub_clf = build_substitute_model()
        stolen_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=True)
        
        attack = ART_attack(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=epochs,
                        nb_stolen=max_queries)
        
        stolen_clf = attack.extract(x_train, thieved_classifier=stolen_clf)

        loss3, acc3 = stolen_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
        loss4, acc4 = stolen_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
        
        clf_results.append((ART_attack.__name__, each_frac, max_queries, epochs,
                            (round(loss1,3), acc1), (round(loss2,3), acc2), 
                            (round(loss3,3), acc3), (round(loss4,3), acc4)))
        
        results_df = pd.DataFrame(clf_results, columns=['Attack', 'Fraction of data', 'Epochs','Max queries', 
                                                         'Train data-victim',
                                                       'Test data-victim', 'Train data-stolen', 
                                                        'Test data-stolen'])
    return results_df

In [35]:
import pandas as pd
KON_results_20e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=5)
KON_results_30e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=5)
KON_results_40e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=5)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Attacking the victim with 5 percent of training data: 2500 queries...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...
Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...
Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...


In [39]:
CCC_results_20e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,30], epoch=5)
CCC_results_30e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,30], epoch=5)
CCC_results_40e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,30], epoch=5)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Attacking the victim with 5 percent of training data: 2500 queries...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 2500 samples
Epoch 1/5
2500/2500 [==============================] - 1s 519us/sample - loss: 2.4546 - accuracy: 0.9048
Epoch 2/5
2500/2500 [==============================] - 1s 448us/sample - loss: 0.3493 - accuracy: 0.9272
Epoch 3/5
2500/2500 [==============================] - 1s 452us/sample - loss: 0.3322 - accuracy: 0.9272
Epoch 4/5
2500/2500 [==============================] - 1s 455us/sample - loss: 0.3213 - accuracy: 0.9272
Epoch 5/5
2500/2500 [==============================] - 1s 467us/sample - loss: 0.2933 - accuracy: 0.9276
Attacking the victim with 10 percent of training data: 5000 queries...
Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 2s 474us/sample - loss: 0.9276 - accuracy: 0.9102
Epoch 2/5
5000/5000 [==============================] - 2s 435us/sample - loss: 0.3319 - accuracy: 0.9202
Epoch 3/5
5000/5000 [==============================] - 2s 434us/sample - loss: 0.3206 - accuracy: 0.9202
Epoch 4/5
5000/5000 [========================

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Train on 2500 samples
Epoch 1/5
2500/2500 [==============================] - 1s 517us/sample - loss: 1.6655 - accuracy: 0.9028
Epoch 2/5
2500/2500 [==============================] - 1s 444us/sample - loss: 0.3068 - accuracy: 0.9292
Epoch 3/5
2500/2500 [==============================] - 1s 445us/sample - loss: 0.3083 - accuracy: 0.9292
Epoch 4/5
2500/2500 [==============================] - 1s 437us/sample - loss: 0.2996 - accuracy: 0.9288
Epoch 5/5
2500/2500 [==============================] - 1s 444us/sample - loss: 0.2922 - accuracy: 0.9296
Attacking the victim with 10 percent of training data: 5000 queries...
Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 2s 481us/sample - loss: 0.9993 - accuracy: 0.9100
Epoch 2/5
5000/5000 [==============================] - 2s 450us/sample - loss: 0.3440 - accuracy: 0.9214
Epoch 3/5
5000/5000 [==============================] - 2s 441us/sample - loss: 0

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Train on 2500 samples
Epoch 1/5
2500/2500 [==============================] - 1s 510us/sample - loss: 1.4428 - accuracy: 0.8980
Epoch 2/5
2500/2500 [==============================] - 1s 439us/sample - loss: 0.3311 - accuracy: 0.9220
Epoch 3/5
2500/2500 [==============================] - 1s 434us/sample - loss: 0.3014 - accuracy: 0.9228
Epoch 4/5
2500/2500 [==============================] - 1s 450us/sample - loss: 0.2872 - accuracy: 0.9240
Epoch 5/5
2500/2500 [==============================] - 1s 446us/sample - loss: 0.2857 - accuracy: 0.9232
Attacking the victim with 10 percent of training data: 5000 queries...
Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 2s 472us/sample - loss: 1.3539 - accuracy: 0.9122
Epoch 2/5
5000/5000 [==============================] - 2s 436us/sample - loss: 0.3166 - accuracy: 0.9254
Epoch 3/5
5000/5000 [==============================] - 2s 436us/sample - loss: 0

In [36]:
EA_results = pd.concat([KON_results_20e,KON_results_30e,KON_results_40e])
EA_results.head(20)

,Attack,Fraction of data,Epochs,Max queries,Train data-victim,Test data-victim,Train data-stolen,Test data-stolen
0,KnockoffNets,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(6.139, 0.099)","(6.156, 0.102)"
1,KnockoffNets,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(9.115, 0.1)","(9.096, 0.1)"
2,KnockoffNets,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(9.775, 0.1)","(9.665, 0.1)"
0,KnockoffNets,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(8.076, 0.1)","(8.072, 0.1)"
1,KnockoffNets,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(6.261, 0.1)","(6.13, 0.101)"
2,KnockoffNets,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(8.365, 0.1)","(8.234, 0.099)"
0,KnockoffNets,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(8.317, 0.1)","(8.354, 0.1)"
1,KnockoffNets,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(8.317, 0.1)","(8.319, 0.1)"
2,KnockoffNets,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(12.114, 0.1)","(11.729, 0.1)"


In [40]:
EA_results = pd.concat([CCC_results_20e,CCC_results_30e,CCC_results_40e])
EA_results.head(10)

,Attack,Fraction of data,Epochs,Max queries,Train data-victim,Test data-victim,Train data-stolen,Test data-stolen
0,CopycatCNN,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(5.005, 0.101)","(4.926, 0.101)"
1,CopycatCNN,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(9.209, 0.1)","(9.026, 0.1)"
2,CopycatCNN,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(8.582, 0.101)","(8.331, 0.099)"
0,CopycatCNN,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(7.656, 0.1)","(7.537, 0.1)"
1,CopycatCNN,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(6.505, 0.1)","(6.345, 0.1)"
2,CopycatCNN,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(9.489, 0.1)","(9.311, 0.1)"
0,CopycatCNN,5,2500,5,"(170.88, 0.098)","(170.812, 0.101)","(8.45, 0.1)","(8.389, 0.1)"
1,CopycatCNN,10,5000,5,"(170.88, 0.098)","(170.812, 0.101)","(9.107, 0.1)","(9.021, 0.1)"
2,CopycatCNN,30,15000,5,"(170.88, 0.098)","(170.812, 0.101)","(10.798, 0.1)","(10.58, 0.101)"


In [12]:
def CCC_extraction_attack(npd_data, x_train, y_train, fractions, epoch=30, verbose=0):
    
    data_size = x_train.shape[0]
    epochs = epoch
    npd_size = npd_data.shape[0]
    
    ### Train and Test sample data from each class to get final results
    print("Extract 100 Training samples from each class:")
    x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
    print("Extract 100 Test samples from each class:")
    x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)
    
    ge_cifar_clf = build_ganeval_model()
    cloud_art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
    
    loss1, acc1 = cloud_art_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
    loss2, acc2 = cloud_art_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
            
    sub_clf = build_substitute_model()
    stolen_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=True)
    
    attack = CopycatCNN(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=epochs,
                        nb_stolen=npd_size)
    
    stolen_clf = attack.extract(npd_data, thieved_classifier=stolen_clf)
    
    stolen_clf._model.save_weights("CCC_temp.h5")
    
    clf_results = []
    
    for each_frac in fractions:

        max_queries = int(data_size*each_frac/100)
        print("Attacking the victim with %d percent of training data: %d queries..."%(each_frac, max_queries))
        
        stolen_clf._model.load_weights("CCC_temp.h5")
        partial_x_train, _ = subset_data(x_train, y_train, fraction=each_frac)

        stolen_clf = attack.extract(partial_x_train, thieved_classifier=stolen_clf)
        
        loss3, acc3 = stolen_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
        loss4, acc4 = stolen_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
        
        clf_results.append(("Copycat CNN", npd_size, each_frac, max_queries, epochs, 
                            (round(loss1,3), acc1), 
                            (round(loss2,3), acc2), 
                            (round(loss3,3), acc3), 
                            (round(loss4,3), acc4)))
        
        results_df = pd.DataFrame(clf_results, columns=['Attack', "NPD Queries", 'Fraction of train data', 
                                                        'Train queries', 
                                                        'Epochs trained', 'Victim clf on Train data',
                                                       'Victim clf on Test data', 'Stolen clf on Train data', 
                                                        'Stolen clf on Test data'])
    return results_df

In [13]:
from tensorflow.keras.datasets import cifar100
(cifar100_x, cifar100_y), _ = cifar100.load_data(label_mode='fine')
print('cifar100 training data shape:', cifar100_x.shape)

# Normalize pixel values
cifar100_x = cifar100_x/255

# Sample 20,000 data points randomly from the data
idx = np.random.choice(cifar100_x.shape[0], 20000, replace=False)
cifar100_x = cifar100_x[idx].copy()
cifar100_y = cifar100_y[idx].copy()

print('Final NPD size of cifar100 training data:', cifar100_x.shape)

169017344/169001437 [==============================] - 3s 0us/step
cifar100 training data shape: (50000, 32, 32, 3)
Final NPD size of cifar100 training data: (20000, 32, 32, 3)


In [14]:
CCC_results_20e = CCC_extraction_attack(cifar100_x, x_train, y_train, [5,10,30], epoch=30)
CCC_results_30e = CCC_extraction_attack(cifar100_x, x_train, y_train, [5,10,30], epoch=30)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


KeyboardInterrupt: ignored

In [ ]:
CCC_results = pd.concat([CCC_results_20e,CCC_results_30e])
CCC_results.head(24)

In [15]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)

(1000, 32, 32, 3) (1000, 10)
(1000, 32, 32, 3) (1000, 10)


In [16]:
partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=0.5)

partial_x_train = partial_x_train.reshape(-1,32*32*3)

### Tried even with standardizing the values around '0'
x_mean = partial_x_train.mean()
x_std = partial_x_train.std()
# partial_x_train = (partial_x_train - x_mean)/x_std

partial_x_train.shape

(250, 32, 32, 3) (250, 10)


(250, 3072)

In [20]:
from keras.models import Sequential
from keras.layers import Activation, Dense
fen_eval_model = Sequential()
fen_eval_model.add(Dense(1024, activation="relu", input_shape=(32*32*3,)))
fen_eval_model.add(Dense(10, activation="linear"))

fen_eval_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,),
    metrics=["accuracy"],
)

In [21]:
fen_eval_model.fit(x_train.reshape(-1,32*32*3), y_train, batch_size=32, epochs=30, verbose=1)

Train on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 8s 160us/sample - loss: 32.8731 - accuracy: 0.2472
Epoch 2/30
50000/50000 [==============================] - 8s 157us/sample - loss: 12.5493 - accuracy: 0.2745
Epoch 3/30
50000/50000 [==============================] - 8s 158us/sample - loss: 8.8341 - accuracy: 0.2823
Epoch 4/30
50000/50000 [==============================] - 8s 155us/sample - loss: 6.2962 - accuracy: 0.2910
Epoch 5/30
50000/50000 [==============================] - 8s 158us/sample - loss: 3.6611 - accuracy: 0.2608
Epoch 6/30
50000/50000 [==============================] - 8s 156us/sample - loss: 2.0776 - accuracy: 0.2470
Epoch 7/30
50000/50000 [==============================] - 8s 157us/sample - loss: 1.9590 - accuracy: 0.2789
Epoch 8/30
50000/50000 [==============================] - 8s 157us/sample - loss: 1.8674 - accuracy: 0.3116
Epoch 9/30
50000/50000 [==============================] - 8s 157us/sample - loss: 1.8254 - accuracy: 0.3247
Epo

In [22]:
fen_eval_model.evaluate(x_test.reshape(-1,32*32*3), y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[1.6728110675811767, 0.3992]

In [23]:
fen_eval_model.evaluate(partial_x_train, partial_y_train)

[1.5862066221237183, 0.44]

In [1]:
flat_art_clf = KerasClassifier(model=fen_eval_model, clip_values=(0, 1), use_logits=False)

attack_FEN = FunctionallyEquivalentExtraction(classifier=flat_art_clf, num_neurons=1024)
stolen_clf_FEN = attack_FEN.extract(partial_x_train, partial_y_train,
                                    delta_0=0.05,
                                    fraction_true=0.2,
                                    rel_diff_slope=0.1,
                                    rel_diff_value=0.1,
                                    delta_init_value=0.1,
                                    delta_value_max=2,
                                    d2_min=0.04,
                                    d_step=0.01,
                                    delta_sign=0.2,
                                    unit_vector_scale=1000)

NameError: ignored

In [ ]:
def build_ganeval_model_flat():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = Sequential()
#     model.add(InputLayer(input_shape=(32*32*3,)))
    model.add(Reshape((32,32,3),input_shape=(32*32*3,)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(Activation('relu'))
        
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [ ]:
ge_flat_clf = build_ganeval_model_flat()
ge_flat_clf.summary()

In [ ]:
ge_flat_clf.evaluate(partial_x_train, partial_y_train)

In [ ]:
flat_art_clf = KerasClassifier(model=ge_flat_clf, clip_values=(0, 1), use_logits=False)

attack_FEN = FunctionallyEquivalentExtraction(classifier=flat_art_clf, num_neurons=100)
stolen_clf_FEN = attack_FEN.extract(partial_x_train, partial_y_train,
                                    delta_0=0.05,
                                    fraction_true=0.3,
                                    rel_diff_slope=0.1,
                                    rel_diff_value=0.1,
                                    delta_init_value=0.1,
                                    delta_value_max=2,
                                    d2_min=0.04,
                                    d_step=0.01,
                                    delta_sign=0.2,
                                    unit_vector_scale=1000)